In [1]:
#importing important packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import pandas as pd
from scipy.special import logsumexp
import requests
from io import BytesIO

In [89]:
#adjacency matrix
A = pd.read_excel(r'/Users/farshidnazemi/Documents/Results.xlsx', sheet_name='Reuse',usecols="D:I", header=24, nrows=6)
A = np.array(A)
A = np.transpose (A)
A = np.float64(A)
A = np.transpose(A)
print (A)
np.linalg.det(A)

[[0. 1. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]]


0.0

In [90]:
#Flow matrix (F)
F = pd.read_excel(r'/Users/farshidnazemi/Documents/Results.xlsx', sheet_name='Reuse',usecols="L:Q", header=24, nrows=6)
F = np.array(F)
F = np.transpose (F)
F = np.float64(F)
F = np.transpose(F)
print (F)

[[ 0.   73.68  0.    0.    0.    4.09]
 [ 0.    0.   70.    0.    0.    0.  ]
 [ 0.    0.    0.   61.67 33.33  0.  ]
 [ 0.    0.    0.    0.    0.   37.  ]
 [ 0.    0.   30.    0.    0.    1.67]
 [ 0.    0.    0.    0.    0.    0.  ]]


In [91]:
#Boundary input vector
Z = pd.read_excel(r'/Users/farshidnazemi/Documents/Results.xlsx', sheet_name='Reuse',usecols="T", header=24, nrows=6)
Z = np.array(Z)
Z = np.transpose (Z)
Z = np.float64(Z)
Z = np.transpose(Z)
Z = Z.flatten()
print (Z) 

[81.87  0.    0.    0.    0.    0.  ]


In [92]:
#Dissipation vector
Y = pd.read_excel(r'/Users/farshidnazemi/Documents/Results.xlsx', sheet_name='Reuse',usecols="W", header=24, nrows=6)
Y = np.array(Y)
Y = np.transpose (Y)
Y = np.float64(Y)
Y = np.transpose(Y)
Y = Y.flatten()
print (Y) 

[ 4.09  3.68  5.   24.67  1.67 42.76]


In [93]:
#Total Flow Matrix
R = pd.read_excel(r'/Users/farshidnazemi/Documents/Results.xlsx', sheet_name='Reuse',usecols="L:S", header=34, nrows=8)
R = np.array(R)
R = np.transpose (R)
R = np.float64(R)
R = np.transpose(R)
print (R)

[[ 0.   73.68  0.    0.    0.    4.09  4.09  0.  ]
 [ 0.    0.   70.    0.    0.    0.    3.68  0.  ]
 [ 0.    0.    0.   61.67 33.33  0.    5.    0.  ]
 [ 0.    0.    0.    0.    0.   37.   24.67  0.  ]
 [ 0.    0.   30.    0.    0.    1.67  1.67  0.  ]
 [ 0.    0.    0.    0.    0.    0.   42.76  0.  ]
 [81.87  0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]]


# Structural-based Metrics

In [94]:
#number of compartments
n=len(A)

In [95]:
#number of links
m=0
for i in range(len(A)):
    for j in range(len(A)):
        if A[i,j]==1:
            m=m+1   
m

8

In [96]:
#Link density
L_d = m/n
L_d

1.3333333333333333

In [97]:
#connectance over direct path
C_d = m/(n**2)
C_d

0.2222222222222222

In [98]:
I = np.identity(n,dtype='float64')
np.linalg.det(I-A)
B = np.linalg.inv(I-A)
print(B)

LinAlgError: Singular matrix

In [99]:
#Total number of direct & indirect paths
m_t=0
for i in range(len(B)):
    for j in range(len(B)):
        if B[i,j]!=0:
            m_t = m_t + B[i,j] 
m_t

NameError: name 'B' is not defined

In [100]:
#Connectance over all paths
C_t = m_t / n**2
C_t

0.0

In [101]:
#pathway proliferation rate
#Find the maximum eigenvalue using NumPy
eigenvalues, eigenvectors = np.linalg.eig(A)
# The eigenvalues array contains the eigenvalues of the matrix
PP = np.max(eigenvalues)
PP

1.0

In [102]:
#Degree diversity
D = np.sum(A, axis=0)
D_dot = np.sum (D)
DD = 0
epsilon = 1e-10  # A very small number
for i in range(n):
    DD += -((D[i] / (D_dot + epsilon)) * (math.log((D[i] + epsilon) / (D_dot + epsilon), 2)))
DD

2.1556390621304864

# Flow-based Metrics

In [103]:
#Total boundary input
TBI = np.sum(Z)
print("Total Boundary Input =", TBI)

Total Boundary Input = 81.87


In [104]:
#Total Internal Flow
TIF = np.sum(F)
print("Total Internal Flow =", TIF)

Total Internal Flow = 311.44000000000005


In [105]:
#Total Input (Output) Throughflows
T_i = np.sum(F, axis=0) + Z
print("Total Input (Output) Throughflows =", T_i)

Total Input (Output) Throughflows = [ 81.87  73.68 100.    61.67  33.33  42.76]


In [106]:
#Flow Analysis (identifies non-dimentional flow intensities along indirect pathways)
G = F/T_i

# Configure printing options for NumPy
np.set_printoptions(precision=2, suppress=True)
# Print the matrix in an organized way
print("Non-dimensional flow intensities (G) =")
print(G)

Non-dimensional flow intensities (G) =
[[0.   1.   0.   0.   0.   0.1 ]
 [0.   0.   0.7  0.   0.   0.  ]
 [0.   0.   0.   1.   1.   0.  ]
 [0.   0.   0.   0.   0.   0.87]
 [0.   0.   0.3  0.   0.   0.04]
 [0.   0.   0.   0.   0.   0.  ]]


In [107]:
#Dimentionless Integral intensities matrix
I=np.identity(n,dtype='float64')
N=np.linalg.inv(I-G)
# Configure printing options for NumPy
np.set_printoptions(precision=2, suppress=True)
# Print the matrix in an organized way
print("Dimentionless Integral intensities matrix (N) =")
print(N)

Dimentionless Integral intensities matrix (N) =
[[1.   1.   1.   1.   1.   1.  ]
 [0.   1.   1.   1.   1.   0.9 ]
 [0.   0.   1.43 1.43 1.43 1.29]
 [0.   0.   0.   1.   0.   0.87]
 [0.   0.   0.43 0.43 1.43 0.43]
 [0.   0.   0.   0.   0.   1.  ]]


In [108]:
#Utility Analysis
D=np.zeros((n,n),dtype='float64')
for i in range (n):
    for j in range (n):
        D[i,j]=(F[j,i]-F[i,j])/(T_i[i]+epsilon)

# Configure printing options for NumPy
np.set_printoptions(precision=2, suppress=True)
# Print the matrix in an organized way
print("Utility Intensities (D) =")
print(D)

Utility Intensities (D) =
[[ 0.   -0.9   0.    0.    0.   -0.05]
 [ 1.    0.   -0.95  0.    0.    0.  ]
 [ 0.    0.7   0.   -0.62 -0.03  0.  ]
 [ 0.    0.    1.    0.    0.   -0.6 ]
 [ 0.    0.    0.1   0.    0.   -0.05]
 [ 0.1   0.    0.    0.87  0.04  0.  ]]


In [109]:
#Utility Intensities
U=np.linalg.inv(I-D)
# Configure printing options for NumPy
np.set_printoptions(precision=2, suppress=True)
# Print the matrix in an organized way
print("Integral Utility Intensities (U) =")
print(U)

Integral Utility Intensities (U) =
[[ 0.62 -0.38  0.25 -0.12 -0.01  0.04]
 [ 0.41  0.42 -0.29  0.11  0.01 -0.09]
 [ 0.22  0.2   0.57 -0.24 -0.01  0.13]
 [ 0.12  0.15  0.36  0.51 -0.02 -0.31]
 [ 0.01  0.02  0.04 -0.04  1.   -0.02]
 [ 0.16  0.09  0.34  0.43  0.02  0.74]]


In [110]:
#Total System Throughflow
TST_flow = np.sum(F)
print("Total System Throughflows (TST_flow) =", TST_flow)

Total System Throughflows (TST_flow) = 311.44000000000005


In [111]:
#Total System Throughput
TST_put = np.sum(R)
print("Total System Throughput (TST_put) =", TST_put)

Total System Throughput (TST_put) = 475.18000000000006


In [112]:
#Average Path Length
APL = TST_flow/(np.sum(Z)+epsilon)
print("Average Path Length (APL) =", APL)

Average Path Length (APL) = 3.804079638446557


In [113]:
#Ratio of direct to indirect flows (I/D)
I_D = np.sum((N-I-G)@T_i)/np.sum(G@T_i)
print("Ratio of direct to indirect flows (I/D) =", I_D)

Ratio of direct to indirect flows (I/D) = 1.9467588345381825


In [114]:
#Indirect effects Index
IEI = I_D/(1+I_D)
print("Indirect effect index (IEI) =", IEI)

Indirect effect index (IEI) = 0.6606440987707362


In [115]:
#Finn's cycling index
#Cycled flow of node i: TSTci
TSTci=np.zeros((n),dtype='float64')
FCI=0.0
for i in range (n):
    TSTci[i]=((N[i,i]-1)/N[i,i])*T_i[i]
for i in range (n):
    FCI=FCI+TSTci[i]/TST_flow
print("Finn's cycling index (FCI) =", FCI)

Finn's cycling index (FCI) = 0.12843244284613406


In [116]:
#Amplification
AMP = 0
# Iterate through the matrix and count elements larger than 1
for row in N:
    for element in row:
        if element > 1:
            AMP += 1

print("Amplification =", AMP)

Amplification = 5


In [117]:
#Amplification percentage
AMP_per = AMP/(n*(n-1))
print("Amplification percentage =", AMP_per)

Amplification percentage = 0.16666666666666666


## Utility Analysis 

In [118]:
U.shape[0]


6

In [119]:
#Network relationships
m = U.shape[0]
# Create a dictionary to store the relationships
relationships = {}

# Process descriptions
process_descriptions = {
    0: "Product Manufacturer (1)",
    1: "Service Provider (2)",
    2: "User/Consumer (3)",
    3: "Collection (EoL) (4)",
    4: "Reuse (5)",
    5: "Ultimate EoL (6)"
    
}

# Iterate through each pair of processes
for i in range(m):
    for j in range(i+1, m):
        Uij = U[i, j]
        Uji = U[j, i]

        if Uij == 0 and Uji == 0:
            relationship = "No Relationship"
        elif Uij > 0 and Uji > 0:
            relationship = "Mutualism"
        elif Uij < 0 and Uji < 0:
            relationship = "Competition"
        else:
            relationship = "Exploitation"

        process_i = process_descriptions[i]
        process_j = process_descriptions[j]

        relationships[f"{process_i} <-> {process_j}"] = relationship

# Print the relationships
for process_pair, relationship in relationships.items():
    print(f"{process_pair}: {relationship}")

Product Manufacturer (1) <-> Service Provider (2): Exploitation
Product Manufacturer (1) <-> User/Consumer (3): Mutualism
Product Manufacturer (1) <-> Collection (EoL) (4): Exploitation
Product Manufacturer (1) <-> Reuse (5): Exploitation
Product Manufacturer (1) <-> Ultimate EoL (6): Mutualism
Service Provider (2) <-> User/Consumer (3): Exploitation
Service Provider (2) <-> Collection (EoL) (4): Mutualism
Service Provider (2) <-> Reuse (5): Mutualism
Service Provider (2) <-> Ultimate EoL (6): Exploitation
User/Consumer (3) <-> Collection (EoL) (4): Exploitation
User/Consumer (3) <-> Reuse (5): Exploitation
User/Consumer (3) <-> Ultimate EoL (6): Mutualism
Collection (EoL) (4) <-> Reuse (5): Competition
Collection (EoL) (4) <-> Ultimate EoL (6): Exploitation
Reuse (5) <-> Ultimate EoL (6): Exploitation


In [120]:
#synergism 
# Calculate the ratio of the sum of positive entries over the sum of negative entries
positive_sum = np.sum(U[U > 0])
negative_sum = np.sum(U[U < 0])

# Avoid division by zero if there are no negative values
if negative_sum != 0:
    syn = positive_sum / abs(negative_sum)
else:
    syn = "Undefined (no negative entries)"

print("Network Synergism:", syn)

Network Synergism: 4.516788706991476


In [121]:
#mutualism
# Calculate the ratio of the number of positive entries over the number of negative entries
positive_count = np.sum(U > 0)
negative_count = np.sum(U < 0)

# Avoid division by zero if there are no negative values
if negative_count != 0:
    mut = positive_count / negative_count
else:
    mut = "Undefined (no negative entries)"

# Print the ratio
print("Network mutualism:", mut)

Network mutualism: 2.272727272727273


In [122]:
#homogenization

# Function to calculate the coefficient of variation (CV)
def coefficient_of_variation(matrix):
    mean = np.mean(matrix)
    std_dev = np.std(matrix)
    
    if mean != 0:
        cv = std_dev / mean
    else:
        cv = "Undefined (mean is zero)"
    
    return cv

# Calculate the CV for matrices G and N
cv_G = coefficient_of_variation(G)
cv_N = coefficient_of_variation(N)

# Calculate the ratio of CV(G) over CV(N)
if cv_N != 0:
    hom = cv_G / cv_N
else:
    hom = "Undefined (CV(N) is zero)"

# Print the ratio
print("Network Homogenization:", hom)

Network Homogenization: 2.608044023041373


In [123]:
#Throughflow diversity
FD = 0
epsilon = 1e-10  # A very small number
for i in range(n):
    FD += -((T_i[i] / (TST_flow + epsilon)) * (math.log((T_i[i] + epsilon) / (TST_flow + epsilon), 2)))
print("Flow Diversity:", FD)

Flow Diversity: 2.725914867556439


## Information Theory 

In [124]:
#Development capacity
DC = 0.0

for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / TST_put + epsilon))

# Print the ascendancy
print("Development Capacity =", DC)

Development Capacity = 1560.3719469307164


In [125]:
#Ascendency
ASC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            ASC += numerator * np.log2(((numerator*TST_put) /denominator)+epsilon)
# Print the ascendancy
print("Ascendency =", ASC)

Ascendency = 1032.4387856545866


In [126]:
#Redundancy
PHI = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            PHI += -numerator * np.log2(((numerator**2) /denominator)+epsilon)

# Print the ascendancy
print("Redundancy =", PHI)

Redundancy = 527.9331609996581


In [127]:
#Ratio of ascendency to development capacity
R_ASC = ASC/DC
print("Relative Ascendency =", R_ASC)

Relative Ascendency = 0.661661975970162


In [128]:
#Internal Development capacity
DC_i = 0.0
for i in range(len(F)):
    for j in range(len(F)):
        if i != j:
            DC_i += -(F[i, j] * np.log2(F[i, j] / TST_put + epsilon))

# Print the ascendancy
print("Internal Development Capacity =", DC_i)

Internal Development Capacity = 998.4937618493648


In [129]:
#Internal ascendency
ASC_i = 0.0
for i in range(len(F)):
    for j in range(len(F)):
        if i != j:
            numerator = F[i, j]
            denominator = np.sum(F[i, :]+ epsilon) * (np.sum(F[:, j]) + epsilon)
            ASC_i += numerator * np.log2(((numerator*TST_put) /denominator)+epsilon)
# Print the ascendancy
print("Internal Ascendency =", ASC_i)

Internal Ascendency = 759.6164285855148


In [130]:
#Internal Redundancy
PHI_i = 0.0
for i in range(len(F)):
    for j in range(len(F)):
        if i != j:
            numerator = F[i, j]
            denominator = np.sum(F[i, :]+ epsilon) * (np.sum(F[:, j]) + epsilon)
            PHI_i += -numerator * np.log2(((numerator**2) /denominator)+epsilon)

# Print the ascendancy
print("Internal Redundancy =", PHI_i)

Internal Redundancy = 238.87733351300304


In [131]:
#Internal relative ascendency
R_ASC_i = ASC_i/DC_i
print("Internal Relative Ascendency =", R_ASC_i)

Internal Relative Ascendency = 0.7607623178121692


In [132]:
#Robustness
Rob = -R_ASC*math.log(R_ASC)
print("Robustness =", Rob)

Robustness = 0.27326670293329214


In [133]:
#Internal Robustness
Rob_i = -R_ASC_i*math.log(R_ASC_i)
print("Internal Robustness =", Rob_i)

Internal Robustness = 0.2080185108233811


## Food web specific metrics

In [134]:
#Detritivory to herbivory ratio
D_H=30/np.sum(Z)
print("Detritivory to herbivory ratio (D:H) =", D_H)

Detritivory to herbivory ratio (D:H) = 0.36643459142543056


In [135]:
#Mixed trophic impact
sFi=np.zeros(n,dtype='float64')
for i in range (n):
    for j in range (n):
        sFi[i]=sFi[i]+F[i,j]
sFj=np.zeros(n,dtype='float64')
for j in range (n):
    for i in range (n):
        sFj[j]=sFj[j]+F[i,j]
QQ=np.zeros((n,n),dtype='float64')
for i in range (n):
    for j in range (n):
        QQ[i,j]=(F[i,j]/(sFj[j]+0.001)-F[j,i]/(sFi[j]+0.001))
#mixed trophic impact
#matrix of the net impacts (qij)
M=np.zeros((n,n),dtype='float64')
M=np.linalg.inv(I-QQ)-I
MTI=np.zeros(n,dtype='float64')
for i in range (n):
    for j in range (n):
        if i!=j:
             MTI[i]=MTI[i]+M[i,j]**2 
    MTI[i]=math.sqrt(MTI[i]) 
MTI=MTI.flatten()
print ("mixed trophic impacts =",MTI)

mixed trophic impacts = [0.48 0.46 0.59 0.44 0.04 0.5 ]


### Based on amount of biomass

In [152]:
#Mean trophic level
#Case 1: Stocks=1 for all compartments
m=8
Num=0.0
Denom=0.0
MTL=0.0
#Trophic levels (TL): 1: virgin from external env. 2: manufacturer 3: service provider 4: consumer 5: open loop recycle 6: recycle 7: redistribute 8: reuse 
BLi=[np.sum(Z),T_i[0],T_i[1],T_i[2],0,0,0,T_i[4]]
TLi=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0,7,8]
for i in range (m):
    Num=Num+BLi[i]*TLi[i]
    Denom=Denom+BLi[i]
MTL=Num/Denom
MTL

3.0567498314227914

# Modified (New) Metrics

In [149]:
#Internal Network Intensity (INI)
INI = TST_flow / T_i[2]
print ("Internal Network Intensity (INI) =", INI)

Internal Network Intensity (INI) = 3.1144000000000007


In [150]:
#Internal Network Intensity (INI)
ONI = TST_put / T_i[2]
print ("Overall Network Intensity (ONI) =", ONI)

Overall Network Intensity (ONI) = 4.7518


In [151]:
#Input Efficiency Indicator
IEFI = T_i[2]/TBI
print ("Input Efficiency Indicator (IEFI) =", IEFI)

Input Efficiency Indicator (IEFI) = 1.2214486380847684
